### Project 2 - Protocol Analysis, Design, and Testing
Dep: Computer Science and Engineering   
Francis J Patron Fidalgo   
Professor Kejie LU   
CIIC 4070 – 090      
Sn: 802 18 0833   

In [1]:
import time
import random
import json
import socket
from queue import Queue 
from threading import Thread, Condition
from ipywidgets import widgets
from IPython.display import display

In [2]:
class ALE_TextInput:
    
    def __init__(self):
        
        self.Queue_User = Queue()
        self.Text = widgets.Text()
        display(self.Text)
        
        def input_handler(sender):
    
            global thread_running
    
            msg = self.Text.value
            print(msg)
            self.Text.value = ""
            if msg=='end':
                thread_running = False

            self.Queue_User.put(msg)
            # for i in range (10):
            #     self.Queue_User.put("{}-{}".format(msg, i))
    
        self.Text.on_submit(input_handler)
        
    # to get a message from user queue. this function can block the thread
    def get(self):
        return self.Queue_User.get()

In [3]:
class ALE_TR:
    
    def __init__(self, name, upper_Tx, lower_TR):
        
        self.Name     = name
        self.Upper_Tx = upper_Tx  # must provide get()
        self.Lower_TR = lower_TR  # must provide send(msg), and receive()
        
    def loop_Tx(self):

        global thread_running
        c = 0
    
        while (thread_running == True):

            c = c + 1

            # get text from Upper_Tx, which must provide a get method
            # this thread is blocked here
            msg = self.Upper_Tx.get()
            print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
            print("%s Tx: message %d: %s"%(self.Name, c, msg))

            # add the text to the queue
            self.Lower_TR.send(msg)
            
    def loop_Rx(self):
        
        global thread_running
        c = 0
    
        while (thread_running == True):

            c = c + 1

            # get message from a lower layer
            # this thread is blocked here
            msg = self.Lower_TR.receive()
            print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
            print("%s Rx: message %d: %s"%(self.Name, c, msg))

In [4]:
# state of DLE
STATE_READY_TO_SEND = 0 # ready to send a packet to the DLE entity in another node through lower layer
STATE_WAITING_ACK   = 1 # waiting for ACK from the DLE entity in another node

# events of DLE
EVENT_UPPER_TX  = 0 # upper layer wants to send a packet
EVENT_LOWER_DAT = 1 # lower layer forwards an incoming data packet
EVENT_LOWER_ACK = 2 # lower layer forwards an incoming data packet
EVENT_TIMEOUT   = 3 # timer timeout
        
class DLE_TR_FSM:
    
    def __init__(self, name, lower_TR):
        
        self.Name      = name
        self.Lower_TR  = lower_TR   # must provide send(msg) and receive()
        self.Queue_Tx  = Queue()
        self.Queue_Rx  = Queue()
        
        # create a queue for Finite State Machine
        self.Queue_FSM = Queue()
        
        # create a flag = condition to start sending
        self.cv_TxOp = Condition()
        self.TxOp    = True

        # init state to STATE_WAITING_UPPER
        self.State = STATE_READY_TO_SEND
        
        self.Procedure = [[self.FSM_upper_Tx, self.FSM_abnormal],
                          [self.FSM_lower_Rx, self.FSM_lower_Rx],
                          [self.FSM_abnormal, self.FSM_lower_Rx_ack],
                          [self.FSM_abnormal, self.FSM_timeout]]
        
        # timer
        self.cv_Timer      = Condition()
        self.timer_active  = False
        self.timer_counter = 0
        
        # transmission 
        self.tx_buffer  = 0
        self.tx_seq_num = 0
        
        # reception
        self.tx_ack_num = 0
        
        
    def loop_Tx(self):

        global thread_running
        c = 0
    
        while (thread_running == True):

            # waiting for the ready to sent signal
            with self.cv_TxOp: 
                while (self.TxOp != True): 
                    self.cv_TxOp.wait()
            
                self.TxOp = False
                
                c = c + 1

                # get a message from queue
                msg = self.Queue_Tx.get()
                print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
                print("%s Tx: message %d: %s"%(self.Name, c, msg))
            
                self.event_add(EVENT_UPPER_TX, msg) 
            
    def loop_Rx(self):
        
        global thread_running
        c = 0
    
        while (thread_running == True):

            c = c + 1

            # get message from a lower layer
            # this thread is blocked here
            msg = self.Lower_TR.receive()
            print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
            print("%s Rx: message %d: %s"%(self.Name, c, msg))
            
            # add an event for FSM, assuming that msg is a sequence of bytes
            msg_type = msg[0]
            if (msg_type == 0):
                self.event_add(EVENT_LOWER_DAT, msg[1:])
            elif (msg_type == 1):
                self.event_add(EVENT_LOWER_ACK, msg[1:])
            else:
                print("%s Rx: message type unknown %d"%(self.Name, msg_type))
            
    def receive(self):
        return self.Queue_Rx.get()
    
    def send(self, msg):
        self.Queue_Tx.put(msg)
        
    def event_add(self, ev_type, msg):
        
        # preparing an event
        if (isinstance(msg, str)):
            msg = msg.encode()
        
        # the event is a sequence of bytes
        event = ev_type.to_bytes(1, "big")+msg
        
        # add event to queue
        self.Queue_FSM.put(event)
        
    def loop_FSM(self):
        
        global thread_running
        
        while (thread_running == True):
            
            # get the next event
            event = self.Queue_FSM.get()
            
            ev_type = event[0]
            
            print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
            print("%s FSM: event type: %d"%(self.Name, ev_type))
            
            # process the event
            msg = event[1:]
            self.Procedure[ev_type][self.State](msg)

    def FSM_abnormal(self, msg):
        
        print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
        print("%s FSM: error! %s"%(self.Name, msg))
    
    def FSM_upper_Tx(self, msg):   
        
        print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
        print("%s FSM: to send frame %d: %s"%(self.Name, self.tx_seq_num, msg.decode('utf-8')))

        # buffer the message
        self.tx_buffer = msg

        # waiting for ACK
        self.State = STATE_WAITING_ACK
        
        # prepare to send a data message
        msg = bytes([0, self.tx_seq_num]) + self.tx_buffer    # inicating a new packet with sequence number
        self.Lower_TR.send(msg)

        # start timer
        with self.cv_Timer:
            self.timer_active  = True
            self.timer_counter = 10
            self.cv_Timer.notify()       
        
        
    
    def FSM_lower_Rx(self, msg):

        # received a new data packet
        seq = msg[0]              # get the sequence number
        msg = msg[1:].decode()
        print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
        print("%s FSM: received frame %d: %s"%(self.Name, seq, msg))

        if (seq == self.tx_ack_num):
            
            # put the message in a receiving queue
            self.Queue_Rx.put(msg)
            
            self.tx_ack_num = self.tx_ack_num + 1
            if (self.tx_ack_num == 256):
                self.tx_ack_num = 0
                
        else:
            print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
            print("%s FSM: received frame %d but expected %d"%(self.Name, seq, self.tx_ack_num))

        # to send an ACK
        self.Lower_TR.send(bytes([1, seq]))

                
    def FSM_lower_Rx_ack(self, msg):

        # received an ACK                
        
        ack = msg[0] # get the ack number
        if (ack != self.tx_seq_num):
            
            # the received ack does not match the seq
            print("ACK # %d does not match local seq # %d"%(ack, self.tx_seq_num))
            return
        
        # stop timer
        self.timer_active  = False
        
        # inc the seq
        self.tx_seq_num = self.tx_seq_num + 1
        if (self.tx_seq_num == 256):
                self.tx_seq_num = 0
        
        # ready to send the next upper layer packet
        with self.cv_TxOp:
            self.State = STATE_READY_TO_SEND
            self.TxOp  = True
            self.cv_TxOp.notify()
            
    def FSM_timeout(self, msg):
        
        print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
        print("%s FSM: to resend frame %d: %s"%(self.Name, self.tx_seq_num, self.tx_buffer.decode('utf-8')))

        # prepare to send a data message
        msg = bytes([0, self.tx_seq_num]) + self.tx_buffer    # inicating a new packet with sequence number
        self.Lower_TR.send(msg)

        # start timer
        with self.cv_Timer:
            self.timer_active  = True
            self.timer_counter = 10
            self.cv_Timer.notify()

    def loop_timer(self):
        
        global thread_running
        
        while (thread_running == True):
            
            with self.cv_Timer: 
                while (self.timer_active == False): 
                    self.cv_Timer.wait() 
            
                time.sleep(0.5)

                if (self.timer_counter == 0):
                    # add an event for timeout
                    self.event_add(EVENT_TIMEOUT, bytes([0]))
                    self.timer_active = False

                else:
                    self.timer_counter = self.timer_counter - 1

In [5]:
class PLE_TR:

    def __init__(self, name, Socket, AP_Tx, AP_Rx):
        
        self.Name      = name
        self.Socket    = Socket
        self.AP_Tx     = AP_Tx
        self.AP_Rx     = AP_Rx
        self.Queue_Tx  = Queue()
        self.Queue_Rx  = Queue()
        
    def loop_Tx(self):
        
        global thread_running
    
        while (thread_running == True):
        
            # get a message from queue
            msg = self.Queue_Tx.get()
            
            # The transmission will be delay after a random amount of time
            # In continous range 0 second to 2 seconds
            time.sleep(random.uniform(0, 2))
        
            print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
            print("%s Tx: %s"%(self.Name, msg))
        
            # The transmission will be lost with probability 10%
            if (random.random() < 0.1):
                print("packet lost")
            else:
                # sending the message using socket
                self.Socket.sendto(msg, self.AP_Tx)
                if (random.random() < 0.05):
                    # The transmission will be duplicated with probability 5%
                    print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
                    print("%s Error: Packet Duplicated"%(self.Name,))
                    self.Socket.sendto(msg, self.AP_Tx)

    def loop_Rx(self):
        
        global thread_running
        global bufferSize
    
        # binding the socket with the IP and port
        self.Socket.bind(self.AP_Rx)
        
        while (thread_running == True):
        
            # get a message from socket, this thread is blocked here
            msg_addr = self.Socket.recvfrom(bufferSize)
    
            msg  = msg_addr[0]
            addr = msg_addr[1]
            
            print(time.strftime("%H:%M:%S", time.localtime()), end=' ')
            print("%s Rx from %s: %s"%(self.Name, addr, msg))        
            self.Queue_Rx.put(msg)
            
    def send(self, msg):
        if (isinstance(msg, str)):
            msg = msg.encode()
        self.Queue_Tx.put(msg)
        
    def receive(self):
        return self.Queue_Rx.get()

In [6]:
class MLE_TR:
    def __init__(self, name, lower_TR, experiment_id, total_msgs, node):
        self.Name     = name
        self.node = node
        self.Lower_TR = lower_TR  # must provide send(msg), and receive()
        self.experiment_id = experiment_id
        self.button = widgets.Button(description=name)
        self.button.on_click(self.button_handler)
        display(self.button)
        self.total_msgs = total_msgs
        # stats
        self.all_delays = [1]
        self.loss_count = 0
        self.duplicates_count = 0
        # trackers
        self.current_msg_id = 0
        self.message_count = 1

    def button_handler(self, rem=False):
        msg = {}
        msg['experiment_id'] = self.experiment_id
        msg['message_id'] = 0
        for i in range(self.total_msgs):
            msg['time'] = time.time_ns()
            self.Lower_TR.send(json.dumps(msg))
            msg['message_id'] += 1 

    def loop_Rx(self):
        global thread_running
        while (thread_running == True):
            msg = json.loads(self.Lower_TR.receive())
            self.message_count += 1
            self.all_delays.append(time.time_ns() - msg.get('time'))
            # check for loss
            msg_delta = msg.get('message_id') - self.current_msg_id
            if msg_delta != 1: # something wrong
                if msg_delta == 0: # repeat
                    self.duplicates_count += 1
                else: # loss
                    self.loss_count += 1
            self.current_msg_id += 1
    
    def stats(self):
        return {
            'exp_id': self.experiment_id,
            'node_id': self.node,
            'avg_delay': sum(self.all_delays) / len(self.all_delays),
            'avg_loss': self.loss_count / self.message_count,
            'avg_duplicate': self.duplicates_count / self.message_count
        }


            
          

In [7]:
thread_running = False
bufferSize = 1024

# (1) create physical layer entities
AP_local_1  = ("127.0.0.1", 30000)
AP_remote_1 = ("127.0.0.1", 31111)
Socket_1    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_1       = PLE_TR("PLE_Alice", Socket_1, AP_remote_1, AP_local_1)

# (2) create date link layer entities
DLE_1 = DLE_TR_FSM("DLE_Alice", PLE_1)

# (3) create application layer entities
ALE_0 = ALE_TextInput()
ALE_1 = ALE_TR("ALE_Alice", ALE_0, DLE_1)

Text(value='')

/tmp/ipykernel_5552/396211819.py:23: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  self.Text.on_submit(input_handler)


In [8]:
# (4) create physical layer entities
AP_local_2  = ("127.0.0.1", 31111)
AP_remote_2 = ("127.0.0.1", 30000)
Socket_2    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_2       = PLE_TR("PLE_Bob", Socket_2, AP_remote_2, AP_local_2)

# (5) create date link layer entities
DLE_2 = DLE_TR_FSM("DLE_Bob", PLE_2)

# (6) create application layer entities
ALE_3 = ALE_TextInput()
ALE_2 = ALE_TR("ALE_Bob", ALE_3, DLE_2)

Text(value='')

/tmp/ipykernel_5552/396211819.py:23: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  self.Text.on_submit(input_handler)


In [9]:
# (7) Measurement Nodes - EXPERIMENT 1
# Node 1 - Physical Layer
AP_local_3  = ("127.0.0.1", 61111)
AP_remote_3 = ("127.0.0.1", 60000)
Socket_3    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_e1n1    = PLE_TR("PLE_Exp1_Node1", Socket_3, AP_remote_3, AP_local_3)
# Node 1 - Measurement Layer
MLE_e1n1    = MLE_TR("MLE_Exp1_Node1", PLE_e1n1, experiment_id=1, total_msgs=100, node=1)
# Node 2 - Physical Layer
AP_local_4  = ("127.0.0.1", 60000)
AP_remote_4 = ("127.0.0.1", 61111)
Socket_4    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_e1n2    = PLE_TR("PLE_Exp1_Node2", Socket_4, AP_remote_4, AP_local_4)
# Node 1 - Measurement Layer
MLE_e1n2    = MLE_TR("MLE_Exp1_Node2", PLE_e1n2, experiment_id=1, total_msgs=100, node=2)

Button(description='MLE_Exp1_Node1', style=ButtonStyle())

Button(description='MLE_Exp1_Node2', style=ButtonStyle())

11:25:55 PLE_Exp1_Node2 Tx: b'{"experiment_id": 1, "message_id": 0, "time": 1679754354644972417}'
11:25:55 PLE_Exp1_Node1 Rx from ('127.0.0.1', 60000): b'{"experiment_id": 1, "message_id": 0, "time": 1679754354644972417}'
11:25:55 PLE_Exp1_Node2 Tx: b'{"experiment_id": 1, "message_id": 1, "time": 1679754354645010224}'
11:25:55 PLE_Exp1_Node1 Rx from ('127.0.0.1', 60000): b'{"experiment_id": 1, "message_id": 1, "time": 1679754354645010224}'
11:25:56 PLE_Exp1_Node2 Tx: b'{"experiment_id": 1, "message_id": 2, "time": 1679754354645019590}'
11:25:56 PLE_Exp1_Node1 Rx from ('127.0.0.1', 60000): b'{"experiment_id": 1, "message_id": 2, "time": 1679754354645019590}'
11:25:58 PLE_Exp1_Node2 Tx: b'{"experiment_id": 1, "message_id": 3, "time": 1679754354645023269}'
packet lost
11:25:59 PLE_Exp1_Node2 Tx: b'{"experiment_id": 1, "message_id": 4, "time": 1679754354645026260}'
11:25:59 PLE_Exp1_Node1 Rx from ('127.0.0.1', 60000): b'{"experiment_id": 1, "message_id": 4, "time": 1679754354645026260}'
11

In [10]:
# (8) Measurement Nodes - EXPERIMENT 2
# Node 1 - Physical Layer
AP_local_5  = ("127.0.0.1", 62222)
AP_remote_5 = ("127.0.0.1", 63333)
Socket_5    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_e2n1    = PLE_TR("PLE_Exp2_Node1", Socket_5, AP_remote_5, AP_local_5)
# Node 1 - Data Link Layer
DLE_e2n1 = DLE_TR_FSM("DLE_Exp2_Node1", PLE_e2n1)
# Node 1 - Measurement Layer
MLE_e2n1    = MLE_TR("MLE_Exp2_Node1", DLE_e2n1, experiment_id=2, total_msgs=100, node=1)
# Node 2 - Physical Layer
AP_local_6  = ("127.0.0.1", 63333)
AP_remote_6 = ("127.0.0.1", 62222)
Socket_6    = socket.socket(family=socket.AF_INET, type=socket.SOCK_DGRAM)
PLE_e2n2    = PLE_TR("PLE_Exp2_Node2", Socket_6, AP_remote_6, AP_local_6)
# Node 1 - Data Link Layer
DLE_e2n2 = DLE_TR_FSM("DLE_Exp2_Node2", PLE_e2n2)
# Node 1 - Measurement Layer
MLE_e2n2    = MLE_TR("MLE_Exp2_Node2", DLE_e2n2, experiment_id=2, total_msgs=100, node=2)

Button(description='MLE_Exp2_Node1', style=ButtonStyle())

Button(description='MLE_Exp2_Node2', style=ButtonStyle())

11:26:56 PLE_Exp1_Node2 Tx: b'{"experiment_id": 1, "message_id": 56, "time": 1679754354645378673}'
11:26:56 PLE_Exp1_Node2 Error: Packet Duplicated
11:26:56 PLE_Exp1_Node1 Rx from ('127.0.0.1', 60000): b'{"experiment_id": 1, "message_id": 56, "time": 1679754354645378673}'
11:26:56 PLE_Exp1_Node1 Rx from ('127.0.0.1', 60000): b'{"experiment_id": 1, "message_id": 56, "time": 1679754354645378673}'
11:26:56 DLE_Exp2_Node2 Tx: message 1: {"experiment_id": 2, "message_id": 0, "time": 1679754416454917273}
11:26:56 DLE_Exp2_Node2 FSM: event type: 0
11:26:56 DLE_Exp2_Node2 FSM: to send frame 0: {"experiment_id": 2, "message_id": 0, "time": 1679754416454917273}
11:26:58 PLE_Exp2_Node2 Tx: b'\x00\x00{"experiment_id": 2, "message_id": 0, "time": 1679754416454917273}'
11:26:58 PLE_Exp2_Node1 Rx from ('127.0.0.1', 63333): b'\x00\x00{"experiment_id": 2, "message_id": 0, "time": 1679754416454917273}'
11:26:58 DLE_Exp2_Node1 Rx: message 1: b'\x00\x00{"experiment_id": 2, "message_id": 0, "time": 1679754

In [11]:
# start the loops of all entities
# all loops must be blocked at a certain position

t1_1 = Thread(target = ALE_1.loop_Tx, args = ()) 
t2_1 = Thread(target = ALE_1.loop_Rx, args = ()) 
t3_1 = Thread(target = DLE_1.loop_Tx, args = ())
t4_1 = Thread(target = DLE_1.loop_Rx, args = ())
t5_1 = Thread(target = PLE_1.loop_Tx, args = ()) 
t6_1 = Thread(target = PLE_1.loop_Rx, args = ())
f1_1 = Thread(target = DLE_1.loop_FSM, args = ())
f2_1 = Thread(target = DLE_1.loop_timer, args = ())

t1_2 = Thread(target = ALE_2.loop_Tx, args = ()) 
t2_2 = Thread(target = ALE_2.loop_Rx, args = ()) 
t3_2 = Thread(target = DLE_2.loop_Tx, args = ())
t4_2 = Thread(target = DLE_2.loop_Rx, args = ())
t5_2 = Thread(target = PLE_2.loop_Tx, args = ()) 
t6_2 = Thread(target = PLE_2.loop_Rx, args = ())
f1_2 = Thread(target = DLE_2.loop_FSM, args = ())
f2_2 = Thread(target = DLE_2.loop_timer, args = ())

ple_e1n1_tx = Thread(target = PLE_e1n1.loop_Tx, args = ()) 
ple_e1n1_rx = Thread(target = PLE_e1n1.loop_Rx, args = ())
ple_e1n2_tx = Thread(target = PLE_e1n2.loop_Tx, args = ()) 
ple_e1n2_rx = Thread(target = PLE_e1n2.loop_Rx, args = ())
ple_e2n1_tx = Thread(target = PLE_e2n1.loop_Tx, args = ()) 
ple_e2n1_rx = Thread(target = PLE_e2n1.loop_Rx, args = ())
ple_e2n2_tx = Thread(target = PLE_e2n2.loop_Tx, args = ()) 
ple_e2n2_rx = Thread(target = PLE_e2n2.loop_Rx, args = ())
dle_e2n1_tx = Thread(target = DLE_e2n1.loop_Tx, args = ())
dle_e2n1_rx = Thread(target = DLE_e2n1.loop_Rx, args = ())
dle_e2n2_tx = Thread(target = DLE_e2n2.loop_Tx, args = ())
dle_e2n2_rx = Thread(target = DLE_e2n2.loop_Rx, args = ())
dle_e2n1_fsm = Thread(target = DLE_e2n1.loop_FSM, args = ())
dle_e2n1_timer = Thread(target = DLE_e2n1.loop_timer, args = ())
dle_e2n2_fsm = Thread(target = DLE_e2n2.loop_FSM, args = ())
dle_e2n2_timer = Thread(target = DLE_e2n2.loop_timer, args = ())
mle_e1n1 = Thread(target = MLE_e1n1.loop_Rx, args = ())
mle_e1n2 = Thread(target = MLE_e1n2.loop_Rx, args = ())
mle_e2n1 = Thread(target = MLE_e2n1.loop_Rx, args = ())
mle_e2n2 = Thread(target = MLE_e2n2.loop_Rx, args = ())


thread_running = True

t1_1.start()
t2_1.start()
t3_1.start()
t4_1.start()
t5_1.start()
t6_1.start()
t1_2.start()
t2_2.start()
t3_2.start()
t4_2.start()
t5_2.start()
t6_2.start()
f1_1.start()
f1_2.start()
f2_1.start()
f2_2.start()

ple_e1n1_tx.start()
ple_e1n1_rx.start()
ple_e1n2_tx.start()
ple_e1n2_rx.start()
ple_e2n1_tx.start()
ple_e2n1_rx.start()
ple_e2n2_tx.start()
ple_e2n2_rx.start()
dle_e2n1_tx.start()
dle_e2n1_rx.start()
dle_e2n2_tx.start()
dle_e2n2_rx.start()
dle_e2n1_fsm.start()
dle_e2n1_timer.start()
dle_e2n2_fsm.start()
dle_e2n2_timer.start()
mle_e1n1.start()
mle_e1n2.start()
mle_e2n1.start()
mle_e2n2.start()


In [12]:
print(t1_1.is_alive())
print(t2_1.is_alive())
print(t3_1.is_alive())
print(t4_1.is_alive())
print(t5_1.is_alive())
print(t6_1.is_alive())

print(t1_2.is_alive())
print(t2_2.is_alive())
print(t3_2.is_alive())
print(t4_2.is_alive())
print(t5_2.is_alive())
print(t6_2.is_alive())

print(f1_1.is_alive())
print(f1_2.is_alive())
print(f2_1.is_alive())
print(f2_2.is_alive())

print(ple_e1n1_tx.is_alive())
print(ple_e1n1_rx.is_alive())
print(ple_e1n2_tx.is_alive())
print(ple_e1n2_rx.is_alive())
print(ple_e2n1_tx.is_alive())
print(ple_e2n1_rx.is_alive())
print(ple_e2n2_tx.is_alive())
print(ple_e2n2_rx.is_alive())
print(dle_e2n1_tx.is_alive())
print(dle_e2n1_rx.is_alive())
print(dle_e2n2_tx.is_alive())
print(dle_e2n2_rx.is_alive())
print(dle_e2n1_fsm.is_alive())
print(dle_e2n1_timer.is_alive())
print(dle_e2n2_fsm.is_alive())
print(dle_e2n2_timer.is_alive())
print(mle_e1n1.is_alive())
print(mle_e1n2.is_alive())
print(mle_e2n1.is_alive())
print(mle_e2n2.is_alive())

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [25]:
output_box = widgets.Output()
display(output_box)
data1 = MLE_e1n1.stats()
data2 = MLE_e2n1.stats()
with output_box:
    output_box.clear_output()
    print(json.dumps(data1, indent=4))
    print(json.dumps(data2, indent=4))

Output()